In [2]:
import numpy as np

import utilities.constants as const
from core.TrajectoryDF import NumPandasTraj as NumTrajDF
from features.spatial_features import SpatialFeatures as spatial
from features.temporal_features import TemporalFeatures as temporal
from utilities.conversions import Conversions as con
import pandas as pd
import time
np.seterr(invalid='ignore')
start = time.time()

In [3]:
%%time

# Reading the geolife dataset and converting to NumPandasTraj.
geolife = pd.read_csv('./data/geolife_sample.csv')
geolife = NumTrajDF(geolife,'lat','lon','datetime','id')
geolife.head()

# Reading the gulls dataset and converting to NumPandasTraj.
gulls = pd.read_csv('./data/gulls.csv')
gulls = NumTrajDF(gulls,
                 latitude='location-lat',
                 longitude='location-long',
                 datetime='timestamp',
                 traj_id='tag-local-identifier',
                 rest_of_columns=[])
#
# Reading the atlantic dataset, cleaning it up and then converting
# it to NumPandasTraj.
atlantic = pd.read_csv('./data/atlantic.csv')
atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)
atlantic = NumTrajDF(atlantic,
                         latitude='Latitude',
                         longitude='Longitude',
                         datetime='DateTime',
                         traj_id='ID',
                         rest_of_columns=[])


CPU times: user 8.9 s, sys: 99.3 ms, total: 9 s
Wall time: 9.01 s


In [4]:
%%time

spatial.get_start_location(atlantic)

CPU times: user 468 ms, sys: 321 ms, total: 789 ms
Wall time: 3.7 s


,lat,lon
traj_id,,
AL011851,28.0,-94.8
AL011852,20.5,-67.1
AL011853,32.5,-69.0
AL011854,26.0,-92.5
AL011855,22.2,-98.0
...,...,...
AL282005,13.2,-62.1
AL291969,10.3,-81.0
AL292005,27.0,-48.0


In [5]:
%%time

spatial.get_end_location(atlantic)

CPU times: user 412 ms, sys: 370 ms, total: 782 ms
Wall time: 3.65 s


,lat,lon
traj_id,,
AL011851,31.0,-100.2
AL011852,41.0,-68.0
AL011853,32.5,-69.0
AL011854,28.0,-103.0
AL011855,22.2,-98.0
...,...,...
AL282005,15.4,-83.5
AL291969,8.5,-82.0
AL292005,35.3,-1.0


In [6]:
%%time

geolife.sort_by_traj_id_and_datetime(True)

CPU times: user 66.9 ms, sys: 15.9 ms, total: 82.8 ms
Wall time: 81.5 ms


,,lat,lon
DateTime,traj_id,,
2008-10-23 05:53:11,1,39.984224,116.319402
2008-10-23 05:53:16,1,39.984211,116.319389
2008-10-23 05:53:21,1,39.984217,116.319422
2008-10-23 05:53:23,1,39.984710,116.319865
2008-10-23 05:53:28,1,39.984674,116.319810
...,...,...,...
2009-03-19 05:46:02,5,39.999896,116.327290
2009-03-19 05:46:07,5,39.999899,116.327352
2009-03-19 05:46:12,5,39.999945,116.327394


In [7]:
%%time

temporal.get_start_time(atlantic)

CPU times: user 382 ms, sys: 238 ms, total: 620 ms
Wall time: 2.59 s


,traj_id
DateTime,
1851-06-25 00:00:00,AL011851
1851-07-05 12:00:00,AL021851
1851-07-10 12:00:00,AL031851
1851-08-16 00:00:00,AL041851
1851-09-13 00:00:00,AL051851
...,...
2015-09-08 06:00:00,AL082015
2015-09-15 00:00:00,AL092015
2015-09-15 12:00:00,AL102015
